In [2]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import numpy as np
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
import pandas as pd
import pickle as pkl
import openai
from numpy import dot
from numpy.linalg import norm

### Batching Tesco Scraping

In [7]:
def get_products(driver):
    """
    Args:
        driver: webdriver
    Returns:
        list of products
    Gets all products from the tesco page and returns them as a list of dictionaries
    """
    count = 0
    products = []
    a_s = driver.find_elements(By.TAG_NAME, "a")
    for a in a_s:
        # check if a has a href attribute
        if(a.get_attribute('href') != None and'/products/' in a.get_attribute('href')):
            product = {}
            count += 1
            #print(f"{count}:{a.get_attribute('href')}")
            try:
                span_child = a.find_element(By.TAG_NAME, "span")
                product = {'name': span_child.text, 'link': a.get_attribute('href'), 'source_page': driver.getCurrentUrl(), 'scrape_time': time.time() }
                gp = a.find_element(By.XPATH, '..')
                gp = gp.find_element(By.XPATH, '..')
                p_elements = gp.find_elements(By.TAG_NAME, "p")
                prices = []
                for p in p_elements:
                    if '£' in p.text:
                        prices.append(p.text)
                product['prices'] = prices
                products.append(product)
            except Exception as e:
                continue
    return products

        


In [5]:
categories = [
    {"name": "fresh-food", "page_count":77},
    {"name": "bakery", "page_count": 15}, 
    {"name": "frozen-food", "page_count": 22}, 
    {"name": "treats-and-snacks", "page_count": 39},
    {"name": "food-cupboard", "page_count": 124},
    {"name": "drinks", "page_count": 66},
    {"name": "household", "page_count": 26},
    {"name": "home-ents", "page_count": 87},
    {"name": "health-and-beauty", "page_count": 80},
    {"name": "baby-and-toddler", "page_count": 19},
    {"name": "pets", "page_count": 16}
]